In [91]:
import numpy as np
from matplotlib import pyplot as plt
from iminuit import Minuit, cost
import pandas as pd

In [92]:
RESONANCE_MASS = 0.775
GAMMA_RESONANCE = 0.149
PION_MASS = 0.140

$$M_{\pi^{+}\pi^{-}} = (E_{\pi^{+}} + E_{\pi^{-}})^{2} - (\vec{p}_{\pi^{+}} + \vec{p}_{\pi^{-}})^{2}$$

In [93]:
# funkcja Källén'a
# fMass - masa pierwszej cząstki
# sMass - masa drugiej cząstki
def Lambda(M_pipi, fMass, sMass):
    return (M_pipi**2 + fMass**2 + sMass **2) - 2*(M_pipi*fMass+M_pipi*sMass+sMass*fMass)

In [94]:
# Gamma w przypadku rezonansu rho(770) zmienia się wraz z masą M_pipi
# q to pęd w układzie środka masy
def Gamma(M_pipi, GAMMA_RESONANCE = GAMMA_RESONANCE):
    def q(M_pipi, pionMass):
        return np.sqrt((M_pipi**2)/4 - pionMass**2)
    return GAMMA_RESONANCE*(q(M_pipi, PION_MASS)/q(RESONANCE_MASS, PION_MASS)**3 * 2./(1.+(q(M_pipi, PION_MASS)/q(RESONANCE_MASS, PION_MASS))**2))


Obie funkcje BW oraz BWM to funkcje Breita-Wignera, BWM jest jej rozszerzoną wersją
gdzie dodatkowo uwzględniana jest temperatura oraz parametr BETA kontrolujący wpływ
temperatury. Gdy za BETE i T damy wartość równą 0 dostaniemy 1 bo jest eksponenta
w efekcie dostaniemy BW

$$M_{R} - \text{masa rezonansu}$$

In [95]:
# Obie funkcje BW oraz BWM to funkcje Breita-Wignera, BWM jest jej rozszerzoną wersją
# gdzie dodatkowo uwzględniana jest temperatura oraz parametr BETA kontrolujący wpływ
# temperatury. Gdy za BETE i T damy wartość równą 0 dostaniemy 1 bo jest eksponenta
# w efekcie dostaniemy BW
def Breit_Wigner(RESONANCE_MASS, GAMMA_RESONANCE, M_pipi, T = 0, BETA = 0):
    return np.exp(BETA*T)/(RESONANCE_MASS**2 - M_pipi**2 - 1j*RESONANCE_MASS*Gamma(M_pipi, GAMMA_RESONANCE))

In [96]:
# A jest jakimś parametrem, Lips - lorentz invariant phase space
def Lips(A, M_pipi):
    return A*np.sqrt(Lambda(M_pipi**2, PION_MASS**2, PION_MASS**2))/M_pipi

In [97]:
# faza Breit-Wignera
def BWPHASE(M_pipi, RESONANCE_MASS, GAMMA_RESONANCE):
    return np.arctan2(Gamma(M_pipi, GAMMA_RESONANCE)*RESONANCE_MASS, (RESONANCE_MASS**2 - M_pipi**2))

# Tpipi czynnik fazowy

def Tpipi(M_pipi, RESONANCE_MASS, GAMMA_RESONANCE):
    return 0.5*(np.exp(2j*BWPHASE(M_pipi, RESONANCE_MASS, GAMMA_RESONANCE))-1.)

In [98]:
def background(M_pipi, B, C, D, RESONANCE_MASS, GAMMA_RESONANCE, E = 1, F = 0, G = 0, T = 0, BETA = 0):
    return(1.+Tpipi(M_pipi, RESONANCE_MASS, GAMMA_RESONANCE))*(B+C*M_pipi)/(1+D*M_pipi)*(E+F*T+G*T**2)*np.exp(BETA*T)

In [99]:
def intensity(M_pipi, A, B, C, D, RESONANCE_MASS, GAMMA_RESONANCE, PHI):
    return Lips(A, M_pipi)*abs(Breit_Wigner(RESONANCE_MASS, GAMMA_RESONANCE, M_pipi)+np.exp(1j*PHI)*background(M_pipi, B, C, D, RESONANCE_MASS, GAMMA_RESONANCE))**2

In [100]:
df=pd.read_csv("moments/Y00-e3.2-3.4.dat",sep="\s+",comment="#",header=None,names=["-t","mpipi","Y00","dY00"])

df=df[df["mpipi"]<=1.2]
 
Y45=df[df["-t"]==0.45]
Y55=df[df["-t"]==0.55]
Y65=df[df["-t"]==0.65]
Y75=df[df["-t"]==0.75]
Y85=df[df["-t"]==0.85]
Y95=df[df["-t"]==0.95]

In [102]:
c = cost.LeastSquares(Y45["mpipi"], Y45["Y00"], Y45["dY00"], intensity)
m1 = Minuit(c, A=0.,B=0.,C=0.,D=0.,RESONANCE_MASS=.8,GAMMA_RESONANCE=.2,PHI=0)
m1.migrad()
m1.minos()

RuntimeError: Function minimum is not valid: <FMin algorithm='Migrad' edm=9.4838802075196 edm_goal=0.0002 errordef=1.0 fval=484.053034552116 has_accurate_covar=False has_covariance=True has_made_posdef_covar=False has_parameters_at_limit=False has_posdef_covar=True has_reached_call_limit=True has_valid_parameters=False hesse_failed=False is_above_max_edm=False is_valid=False nfcn=1149 ngrad=0 reduced_chi2=6.630863487015287 time=0.125>